## Step 3: Optimise the Model
Using your knowledge of TensorFlow, optimize your model to achieve a target predictive accuracy higher than 75%.

Using any or all of the following methods to optimize your model:

* Adjust the input data to ensure that no variables or outliers are causing confusion in the model, such as:
  * Dropping more or fewer columns.
  * Creating more bins for rare occurrences in columns.
  * Increasing or decreasing the number of values for each bin.
* Add more neurons to a hidden layer.
* Add more hidden layers.
* Use different activation functions for the hidden layers.
* Add or reduce the number of epochs to the training regimen.

***Note**: If you make at least three attempts at optimizing your model, you will not lose points if your model does not achieve target performance.*

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

2023-02-08 21:41:42.146747: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Preprocess the dataset like you did in Step 1, Be sure to adjust for any modifications that came out of optimizing the model.


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 100
application_types_to_replace = []
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(
    classification_count[classification_count<1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 42)
    # Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model
Design a neural network model, and be sure to adjust for modifications that will optimize the model to achieve higher than 75% accuracy.

## Optimisation Test 1

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_1 = tf.keras.models.Sequential()
input_dim = X_train_scaled.shape[1]
# Input layer
nn_1.add(tf.keras.layers.Dense(
    units = 16, activation = "relu", input_dim = input_dim))

# First hidden layer
nn_1.add(tf.keras.layers.Dense(
    units = 20, activation = "relu"))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(
    units = 12, activation = "relu"))

# Output layer
nn_1.add(tf.keras.layers.Dense(
    units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                832       
                                                                 
 dense_1 (Dense)             (None, 20)                340       
                                                                 
 dense_2 (Dense)             (None, 12)                252       
                                                                 


2023-02-08 21:41:52.618169: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 dense_3 (Dense)             (None, 1)                 13        
                                                                 
Total params: 1,437
Trainable params: 1,437
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn_1.compile(
    loss = "binary_crossentropy", 
    optimizer = "adam", 
    metrics = ["accuracy"]
    )
# Train the model
fit_model = nn_1.fit(
    X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5843 - accuracy: 0.7121
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5584 - accuracy: 0.7281
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5518 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7320
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7323
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7318
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7315
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accura

In [13]:
# Evaluate the model using the test data
model_loss1, model_accuracy1 = nn_1.evaluate(
    X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss1}, Accuracy: {model_accuracy1}")

268/268 - 1s - loss: 0.5617 - accuracy: 0.7291 - 770ms/epoch - 3ms/step
Loss: 0.5616807341575623, Accuracy: 0.7290962338447571


In [14]:
# Export our model to HDF5 file
nn_1.save('./Output/AlphabetSoupCharity_Optimisation1.h5')

## Optimisation Test 2

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_2 = tf.keras.models.Sequential()
input_dim = X_train_scaled.shape[1]
# Input layer
nn_2.add(tf.keras.layers.Dense(
    units = 12, activation = "relu", input_dim = input_dim))

# First hidden layer
nn_2.add(tf.keras.layers.Dense(
    units = 16, activation = "relu"))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(
    units = 20, activation = "relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(
    units = 16, activation = "relu"))

# Fourth hidden layer
nn_2.add(tf.keras.layers.Dense(
    units = 12, activation = "relu"))

# Fifth hidden layer
nn_2.add(tf.keras.layers.Dense(
    units = 8, activation = "relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(
    units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 12)                624       
                                                                 
 dense_5 (Dense)             (None, 16)                208       
                                                                 
 dense_6 (Dense)             (None, 20)                340       
                                                                 
 dense_7 (Dense)             (None, 16)                336       
                                                                 
 dense_8 (Dense)             (None, 12)                204       
                                                                 
 dense_9 (Dense)             (None, 8)                 104       
                                                                 
 dense_10 (Dense)            (None, 1)                

In [16]:
# Compile the model
nn_2.compile(
    loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Train the model
fit_model = nn_2.fit(
    X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 5s 3ms/step - loss: 0.5924 - accuracy: 0.7037
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5592 - accuracy: 0.7267
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5557 - accuracy: 0.7284
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5530 - accuracy: 0.7293
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5521 - accuracy: 0.7311
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5508 - accuracy: 0.7305
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5496 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5497 - accuracy: 0.7311
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5488 - accuracy: 0.7304
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5480 - accura

In [17]:
# Evaluate the model using the test data
model_loss2, model_accuracy2 = nn_2.evaluate(
    X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

268/268 - 1s - loss: 0.5567 - accuracy: 0.7279 - 887ms/epoch - 3ms/step
Loss: 0.5566837191581726, Accuracy: 0.7279300093650818


In [18]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn_2.save('./Output/AlphabetSoupCharity_Optimisation2.h5')

## Optimisation Test 3

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_3 = tf.keras.models.Sequential()
input_dim = X_train_scaled.shape[1]

# Input layer
nn_3.add(tf.keras.layers.Dense(
    units = 12, activation = "relu", input_dim = input_dim))

# First hidden layer
nn_3.add(tf.keras.layers.Dense(
    units = 32, activation = "relu"))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(
    units = 40, activation = "relu"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(
    units = 32, activation = "relu"))

# Fourth hidden layer
nn_3.add(tf.keras.layers.Dense(
    units = 24, activation = "relu"))

# Fifth hidden layer
nn_3.add(tf.keras.layers.Dense(
    units = 16, activation = "relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(
    units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 12)                624       
                                                                 
 dense_12 (Dense)            (None, 32)                416       
                                                                 
 dense_13 (Dense)            (None, 40)                1320      
                                                                 
 dense_14 (Dense)            (None, 32)                1312      
                                                                 
 dense_15 (Dense)            (None, 24)                792       
                                                                 
 dense_16 (Dense)            (None, 16)                400       
                                                                 
 dense_17 (Dense)            (None, 1)                

In [20]:
# Compile the model
nn_3.compile(
    loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
# Train the model
fit_model = nn_3.fit(
    X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 6s 3ms/step - loss: 0.5855 - accuracy: 0.7077
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5587 - accuracy: 0.7264
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5538 - accuracy: 0.7273
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7295
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7294
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5474 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5460 - accura

In [21]:
# Evaluate the model using the test data
model_loss3, model_accuracy3 = nn_3.evaluate(
    X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

268/268 - 1s - loss: 0.5618 - accuracy: 0.7269 - 640ms/epoch - 2ms/step
Loss: 0.5617533922195435, Accuracy: 0.7268804907798767


In [22]:
# Export our model to HDF5 file
nn_3.save('./Output/AlphabetSoupCharity_Optimisation3.h5')

## Auto Optimisation

In [23]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    input_dim = X_train_scaled.shape[1]
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(
        units = hp.Int('first_units',
        min_value = 1,
        max_value = 80,
        step = 5), 
        activation = activation, 
        input_dim =  input_dim))
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units = hp.Int('units_' + str(i),
            min_value = 1,
            max_value = 100,
            step = 5),
            activation = activation))
    nn_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))
    # Compile the model
    nn_model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ["accuracy"])
    
    return nn_model

In [24]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    overwrite = True,
    max_epochs = 20,
    hyperband_iterations = 2)

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs = 20, validation_data = (X_test_scaled,y_test))

Trial 60 Complete [00h 01m 00s]
val_accuracy: 0.7323614954948425

Best val_accuracy So Far: 0.7336443066596985
Total elapsed time: 00h 42m 29s
INFO:tensorflow:Oracle triggered exit


In [25]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 71,
 'num_layers': 3,
 'units_0': 16,
 'units_1': 51,
 'units_2': 66,
 'units_3': 71,
 'units_4': 76,
 'units_5': 31,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [26]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5554 - accuracy: 0.7336 - 778ms/epoch - 3ms/step
Loss: 0.5553974509239197, Accuracy: 0.7336443066596985


In [27]:
# Export our model to HDF5 file
best_model.save('./Output//AlphabetSoupCharity_autoOptimisation.h5')

# Best Model
Loss: 0.5553974509239197, Accuracy: 0.7336443066596985

`{
    'activation': 'sigmoid',
    'first_units': 71,
    'num_layers': 3,
    'units_0': 16,
    'units_1': 51,
    'units_2': 66,
    'units_3': 71,
    'units_4': 76,
    'units_5': 31,
    'tuner/epochs': 20,
    'tuner/initial_epoch': 0,
    'tuner/bracket': 0,
    'tuner/round': 0
}`

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_best = tf.keras.models.Sequential()
input_dim = X_train_scaled.shape[1]

# Input layer
nn_best.add(tf.keras.layers.Dense(
    units = 16, activation = "relu", input_dim = input_dim))

# First hidden layer
nn_best.add(tf.keras.layers.Dense(
    units = 51, activation = "relu"))

# Second hidden layer
nn_best.add(tf.keras.layers.Dense(
    units = 66, activation = "relu"))

# Third hidden layer
nn_best.add(tf.keras.layers.Dense(
    units = 71, activation = "relu"))

# Fourth hidden layer
nn_best.add(tf.keras.layers.Dense(
    units = 76, activation = "relu"))

# Fifth hidden layer
nn_best.add(tf.keras.layers.Dense(
    units = 31, activation = "relu"))

# Output layer
nn_best.add(tf.keras.layers.Dense(
    units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_best.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 16)                832       
                                                                 
 dense_13 (Dense)            (None, 51)                867       
                                                                 
 dense_14 (Dense)            (None, 66)                3432      
                                                                 
 dense_15 (Dense)            (None, 71)                4757      
                                                                 
 dense_16 (Dense)            (None, 76)                5472      
                                                                 
 dense_17 (Dense)            (None, 31)                2387      
                                                                 
 dense_18 (Dense)            (None, 1)                

In [31]:
# Compile the model
nn_best.compile(
    loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
# Train the model
fit_model = nn_best.fit(
    X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5756 - accuracy: 0.7173
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7300
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accuracy: 0.7296
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7325
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5480 - accuracy: 0.7314
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5465 - accuracy: 0.7329
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5467 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_best.evaluate(
    X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
# Export our model to HDF5 file
nn_best.save('./Output/AlphabetSoupCharity_bestOptimisation.h5')

268/268 - 1s - loss: 0.5822 - accuracy: 0.7268 - 628ms/epoch - 2ms/step
Loss: 0.5821629762649536, Accuracy: 0.7267638444900513
